Integração e processamento de dados para estudo de gravidade. 

Autoras: Maria Gabriela Valeriano e Ana Carolina Lorena

Dados disponíveis em: https://repositoriodatasharingfapesp.uspdigital.usp.br/

Os arquivo contendo os exames passou por um processamento anterior (notebook: bp_nomesexames)

In [1]:
#carrega o pacote tidyverse
library(tidyverse)

Registered S3 methods overwritten by 'tibble':
  method     from  
  format.tbl pillar
  print.tbl  pillar

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang

Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──

✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
#lê o arquivo pacientes do bp
pacientes <- read.csv("bpsp_pacientes_01.csv", sep = "|", stringsAsFactors = FALSE)

#dimensões
pacientes %>% dim

#verifica se tem linhas repetidas
pacientes %>% unique %>% dim


[1] 39000     7

[1] 39000     7

In [3]:
#renomeia as colunas
names(pacientes) <- c('id', 'sexo', 'nascimento.ano', 'país', 'estado', 'cidade', 'CEP')

#número de pacientes
pacientes %>% select(id) %>% unique %>% nrow

[1] 39000

In [4]:
#le o arquivo bp exames - com nomes processados
exames <- read.csv("bpsp_exames_01_nomesproc.csv", stringsAsFactors = FALSE)

#dimensões
exames %>% dim

#verifica se tem linhas repetidas
exames %>% unique %>% dim

#remove as linhas repetidas
exames %>% unique -> exames

#numero de ids no arquivo
exames %>% select(id) %>% unique %>% dim


[1] 5339293      10

[1] 5229119      10

[1] 33800     1

In [5]:
#transforma as vírgulas em pontos em todo o arquivo
exames <- data.frame(lapply(exames, function(x) {gsub(",", ".", x)}))


In [6]:
#lê o arquivo desfecho do bp
desfecho <- read.csv("bpsp_desfecho_01.csv", sep = "|", stringsAsFactors = FALSE) 

#dimensões
desfecho %>% dim

#verifica se tem linhas repetidas
desfecho %>% unique %>% dim

[1] 217991      8

[1] 217991      8

In [7]:
#renomeia as colunas
names(desfecho) <- c('id', 'id.atendimento', 'data.atendimento', 'tipo.atendimento', 'id.clinica', 'descrição', 'data.desfecho','desfecho.paciente')

#número de dis no arquivo
desfecho %>% select(id) %>% unique %>% dim

[1] 39001     1

In [8]:
#passa as coluna data.desfecho e data.atendimento para o formato data
desfecho$data.desfecho <- as.Date(desfecho$data.desfecho, format="%d/%m/%Y")
desfecho$data.atendimento<- as.Date(desfecho$data.atendimento, format="%d/%m/%Y")

#adiciona uma coluna com o número de dias entre entrada e a data do desfecho
desfecho %>% mutate(dias.desfecho = (difftime(desfecho$data.desfecho, desfecho$data.atendimento, unit = "days"))) -> desfecho
desfecho$dias.desfecho <- as.integer(desfecho$dias.desfecho)


In [9]:
#faz join dos três arquivos
exames %>% left_join(pacientes) -> dados1
dados1 %>% left_join(desfecho) -> dados

#verifica quantos pacientes (ids) estão presentes
dados %>% select(id) %>% unique %>% dim

#dimensões do arquivo (uma linha por exame)
dados %>% dim

Joining, by = "id"

Warning message:
“Column `id` joining factor and character vector, coercing into character vector”
Joining, by = c("id", "id.atendimento")

Warning message:
“Column `id.atendimento` joining factor and character vector, coercing into character vector”


[1] 33800     1

[1] 5229119      23

In [10]:
#passa as colunas data.exame e data.atendimento para o formato data
dados$data.exame <- as.Date(dados$data.exame, format="%d/%m/%Y")
dados$data.atendimento<- as.Date(dados$data.atendimento, format="%d/%m/%Y")

#acrescenta a coluna dias exame (depois de quantos dias após a entrada fez o exame)
dados %>% mutate(dias.exame = (difftime( dados$data.exame, dados$data.atendimento,  unit = "days"))) -> dados
dados$dias.exame <- as.integer(dados$dias.exame)

#resultado 
dados %>% dim
dados -> exames

[1] 5229119      24

In [13]:
#filtra Covid na coluna exame
exames %>% 
filter (str_detect(exame,"covid") & !str_detect(exame, "pre") & !str_detect(exame,"inativo")) %>% 
count(exame)

exame,n
<fct>,<int>
coronavirus covid19,33965


In [14]:
#filtra Covid na coluna exame - salva
exames %>% 
filter (str_detect(exame, "covid") & !str_detect(exame, "pre") & !str_detect(exame, "inativo")) -> exame_pcr

#verifica os analitos desses exames
exame_pcr %>% count(analito)

analito,n
<fct>,<int>
coronavirus 2019ncov,33960
covid 19 cultura,5


In [15]:
#filtra o analito de interesse
exame_pcr %>% filter (str_detect(analito, "2019"))  %>% count(analito)

#filtra o analito de interesse - salva
exame_pcr %>% filter (str_detect(analito, "2019"))  -> exames_pcr_covid

analito,n
<fct>,<int>
coronavirus 2019ncov,33960


In [16]:
#nos interessam os resultados positivos
exames_pcr_covid %>% count(resultado)

#filtra os pacientes que testaram positivo
exames_pcr_covid %>% filter (str_detect(resultado, "DETECTADO") & !str_detect(resultado, "NAO")) %>% count(resultado)

resultado,n
<fct>,<int>
DETECTADO,993
DETECTADO (POSITIVO),6140
DETECTAVEL,15
Inconclusivo,11
INCONCLUSIVO,34
Inconclusivo,8
INDETECTAVEL,198
NAO DETECTADO,2206
NAO DETECTADO (NEGATIVO),24355


resultado,n
<fct>,<int>
DETECTADO,993
DETECTADO (POSITIVO),6140


In [17]:
#filtra os pacientes que testaram positivo - salva
exames_pcr_covid %>% filter (str_detect(resultado, "DETECTADO") & !str_detect(resultado, "NAO")) -> exames_pcr_covid_positivo

In [18]:
#filtra os pacientes que testaram positivo em no máximo 15 dias

#transforma a coluna dias.exame em numérica
exames_pcr_covid_positivo$dias.exame <- as.numeric(exames_pcr_covid_positivo$dias.exame)

#filtra os exames feitos em no máximo 15 dias
exames_pcr_covid_positivo %>% filter(dias.exame <16) -> exames_pcr_covid_positivo

#verifica se o filtro funcionou como esperado
exames_pcr_covid_positivo %>% count(dias.exame)

#seleciona os ids.atendimento dos pacientes que testaram positivo em no máximo 15 dias
exames_pcr_covid_positivo %>%
select(id.atendimento) %>% unique -> id_positivo

#número de ids.atendimento que testaram positivo em no máximo 15 dias
id_positivo %>% tally


dias.exame,n
<dbl>,<int>
0,6411
1,454
2,84
3,27
4,8
5,7
6,9
7,5
8,5


n
<int>
7022


In [19]:
#filtra todos os exames feitos pelos ids.atendimento positivos
exames %>% filter(id.atendimento %in% id_positivo$id.atendimento) -> exames

#dimensão após filtro
exames %>% dim


[1] 552294     24

In [20]:
#verifica quantos pacientes (ids) e em quantos atendimentos (id.atendimento) 
exames %>% select(id) %>% unique %>% dim
exames %>% select(id.atendimento) %>% unique %>% dim

[1] 6786    1

[1] 7022    1

In [21]:
#filtra os exames feitos até o quarto dia após atendimento
exames %>% filter(dias.exame == "0" | dias.exame == "1" | dias.exame == "2" | dias.exame == "3" | dias.exame == "4") -> exames

#cria mais uma coluna unindo exameEanalito
exames %>% mutate(exameEanalito = (str_c(exame, "/", analito))) -> exames

#verifica quantos pacientes (ids) e em quantos atendimentos (id.atendimento) 
exames$id.atendimento %>% unique() %>% length()
exames$id %>% unique() %>% length()

#dimensão dos dados após o filtro
exames %>% dim

#verifica se o filtro funcionou como esperado
exames %>% count(dias.exame)

[1] 7021

[1] 6786

[1] 200161     24

In [ ]:
#alguns exames foram feitos mais de uma vez dentro do mesmo id de atendimento, remove ficando com o último resultado
exames %>% group_by(id.atendimento,  exameEanalito) %>% arrange((dias.exame)) %>% slice(1) %>% ungroup -> exames

#dimensão dos dados após o filtro
exames %>% dim 

In [ ]:
#nomes das colunas
exames %>% colnames

#tipos de desfecho
exames %>% count(desfecho.paciente)

#quantos pacientes foram internados mais de 9 dias
exames %>% filter (tipo.atendimento == 'Internado') %>% filter(dias.desfecho > 9) %>% dim

In [ ]:
#cria essas variáveis para verificar se a gravidade foi atribuida corretamente
obito_1 <- 0
obito_2 <- 0
obito_3 <- 0
obito_4 <- 0
internado <- 0

#adiciona uma coluna com a gravidade = não
exames %>% mutate(gravidade = 0) -> exames


#condições de gravidade
for (i in (1:nrow(exames))){ #percorre todo o arquivo
  dias <- exames$dias.desfecho[i]
  desfpac <- exames$desfecho.paciente[i]

  #se paciente internado e mais de 9 dias entre a entrada e o desfecho
  if (!(is.na(dias)) && exames$tipo.atendimento[i] == "Internado" && dias > 9){ 
   exames$gravidade[i] <- 1
   internado = internado+1}

  
  #se óbito
  if (!(is.na(desfpac)) &
      desfpac == "Obito com declaracao fornecida pelo medico assistente Maior que 24 Horas de inte")
  {   exames$gravidade[i] <- 1 
      obito_1 = obito_1+1}

  if (!(is.na(desfpac)) &
      desfpac == "Obito com declaracao fornecida pelo medico assistente Menor que 24 Horas de inte")
  {   exames$gravidade[i] <- 1  
      obito_2 = obito_2+1}

  if (!(is.na(desfpac)) &
      desfpac == "Obito com declaracao fornecida pelo- IML Maior que 24 Horas de internacao")
  {   exames$gravidade[i] <- 1 
      obito_3 = obito_3+1}

  if (!(is.na(desfpac)) &
      desfpac == "Obito com declaracao fornecida pelo- SVO Maior que 24 Horas de internacao")
  {   exames$gravidade[i] <- 1 
      obito_4 = obito_4+1}
}

#verifica se gravidade foi atribuída corretamente
exames %>% count(gravidade)
obito_1
obito_2
obito_3
obito_4
internado

In [ ]:
#dimensão dos dados 
exames %>% dim

#acrescenta a coluna idade
exames %>% mutate(idade = (as.numeric(format(as.Date(exames$data.atendimento), '%Y')) -  
                as.numeric(exames$nascimento.ano))) -> exames

exames %>% count(idade)

#transforma sexo em númerico
as.character(exames$sexo) -> exames$sexo
exames$sexo[exames$sexo == "F"] <- 1
exames$sexo[exames$sexo == "M"] <- 0
as.numeric(exames$sexo) -> exames$sexo
exames %>% count(sexo)

In [ ]:
exames %>% colnames
exames %>% ncol

In [ ]:
#pivota transformando cada id.atendimento em uma linha
exames %>%   select(c(1,2,5,10,11,17,26,25)) %>% #seleciona as colunas id, exame+analito e resultado, idade, sexo, gravidade
  group_by('id.atendimento') %>% #agrupa por id
  pivot_wider(names_from = exameEanalito, values_from = resultado) %>%  #pivota transformando exame+analito em várias colunas
  ungroup -> pivotado 

#quantos atendimentos são graves
pivotado %>% count(gravidade)

#nova dimensão dos dados
pivotado %>% dim


In [ ]:
#remove os pacientes que não fizeram hemograma - verificado pelo analito neutrófilos
pivotado = pivotado[!is.na(pivotado$'hemograma/neutrofilos'),  ]

#dimensão dos dados após o filtro
pivotado %>% dim

#quantos são graves
pivotado %>% count(gravidade)

#quantos pacientes há na base
pivotado %>% select(id) %>% unique %>% dim

In [ ]:
#filtra apenas um id.atendimento por paciente
pivotado %>% group_by(id) %>% arrange(desc(data.atendimento)) %>% slice(1) %>% ungroup -> pivotado

#dimensão dos dados após o filtro
pivotado %>% dim

#quantos são graves
pivotado %>% count(gravidade)

pivotado -> exames

In [ ]:
#quais datas foram abarcadas nesse intervalo
pivotado %>% count(data.atendimento)

In [ ]:
#algumas exames contém valores não númericos
exames <- data.frame(lapply(exames, function(x) {gsub("superior a", "", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "inferior a", "", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "1.000.000.", "1000000", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "g/L", "", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "1.000.000", "1000000", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "negativa", "0", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "Parcial Positivo", "1", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "Parcial Nositivo", "0", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "POSITIVO", "1", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "NEGATIVO", "0", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "NÃO REAGENTE", "0", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "REAGENTE", "1", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "Indeterminado", "NA", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "não reagente", "0", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "REAGENTE. maior ou igual a 1000", "1000", x)}))
exames <- data.frame(lapply(exames, function(x) {gsub( "Parcial Negativo", "0", x)}))

In [ ]:
#transforma o formato das colunas
dados <- as.numeric(exames)


In [ ]:
#quantas e quais são as colunas não numéricas
dados %>% dplyr::select(where(is.character)) %>% ncol

dados %>% dplyr::select(where(is.character)) %>% colnames

In [ ]:
#filtra apenas as colunas numéricas
dados %>% dplyr::select(where(is.numeric)) -> dados

#resultado
dados %>% dim
dados %>% count(gravidade)

In [ ]:
#filtra apenas os pacientes graves
dados %>% filter(gravidade == 1) -> graves

#filtra os exames que ao menos 50% de valores não ausentes nesta classe

colunas = c()
#para cada coluna 
for (i in 1:ncol(dados)) { 
  #conta os valores diferentes de NA
  temp = sum(!is.na(graves[,i]))
  #se os valores diferentes de NA forem mais da metade do total de graves seleciona essa coluna
  if (temp >= (nrow(graves)/2)) {colunas <- append(colunas, i)}
}

#filtra as colunas selecionadas
dados %>% select(all_of(colunas)) -> dados

#número de colunas
dados %>% ncol


In [ ]:
#filtra apenas os pacientes não graves
dados %>% filter(gravidade == 0) -> ngraves

#filtra os exames que ao menos 50% de valores não ausentes nesta classe

colunas = c()
#para cada coluna 
for (i in 1:ncol(dados)) { 
    #conta os valores diferentes de NA
    temp = sum(!is.na(ngraves[,i]))
    #se os valores diferentes de NA forem mais da metade do total de ngraves seleciona essa coluna
    if (temp >= (nrow(ngraves)/2)) {colunas <- append(colunas, i)}
}

#filtra as colunas selecionadas
dados %>% select(all_of(colunas)) -> dados

#número de colunas
dados %>% ncol

In [ ]:
#conjunto de funções para remover valores extremos, foram removidos 4 valores extremos por variável
#a decisão por remover valores extremos à direita, à esquerda ou de ambas foi feita a partir de análise visual dos histogramas

#biblioteca usada para transformar os valores em na
library(naniar)

#função para transformar em NA os valores extremos da direita
re_extremos_dir <- function(column) {

  #a função map passa as colunas como listas
  #transforma em tibble e acrescenta um "nome"
  column %>% as.tibble -> column
  colnames(column)[1] <- "label" 

  #define os 4 valores extremos à direita
  column %>% na.omit %>% count(label) %>% slice(tail(row_number(), 4)) %>% select(1) %>% as.list -> limite
  limite[[1]] ->> limite

  #transforma esses valores em NA
  column %>% replace_with_na_all(condition = ~.x %in% limite) -> column
  
} 

#função para transformar em NA os valores extremos da esquerda
re_extremos_esq <- function(column) {

  #a função map passa as colunas como listas
  #transforma em tibble e acrescenta um "nome"
  column %>% as.tibble -> column
  colnames(column)[1] <- "label" 

  #define os 4 valores extremos à esquerda
  column %>% count(label) %>% slice(1:4) %>% select(1) %>% as.list -> limite
  c(limite[[1]]) ->> limite

  #transforma esses valores em NA
  column %>% replace_with_na_all(condition = ~.x %in% limite) -> column

  
}

#função para transformar em NA os valores extremos da esquerda e da direita
re_extremos_dir_esq <- function(column) {

  #a função map passa as colunas como listas
  #transforma em tibble e acrescenta um "nome"
  column %>% as.tibble -> column
  colnames(column)[1] <- "label" 

  #define os 2 valores extremos à esquerda
  column %>% count(label) %>% slice(1:2) %>% select(1) %>% as.list -> limite
  c(limite[[1]]) ->> limite

  #define os 2 valores extremos à direita
  column %>% na.omit %>% count(label) %>% slice(tail(row_number(), 2)) %>% select(1) %>% as.list -> limite2
  c(limite2[[1]]) ->> limite2

  #transforma esses valores em NA
  column %>% replace_with_na_all(condition = ~.x %in% limite2) -> column
  column %>% replace_with_na_all(condition = ~.x %in% limite) -> column

}


In [ ]:
library(ggplot2)

for (i in (1:ncol(dados))){

  label <- (dados[i] %>% colnames)
  #label <- label %>% str_replace(".", "")
  exame <- dados %>% select(i) %>% as.list
  exame[[1]] -> exame
  #b <-(max(exame)-min(exame))/50
  His <- ggplot(data=dados, 
  aes(x=exame))+
  geom_histogram(bins = 50, color="black", aes(fill=factor(gravidade))) +  
  xlab(label) +   ylab("Frequencia") +      
  theme(legend.position="none")+  ggtitle(paste("Histograma", label))+      
  geom_vline(data=dados, aes(xintercept = median(exame)),linetype="dashed",color="grey")
  show(His)
}


In [ ]:
#variáveis com histograma dislocado à esquerda, remover valores extremos da direita
dados %>% select('creatinina.plasma.creatinina',
                 'hemograma.basofilos',
                 'hemograma.basofilos.percentual',
                 'hemograma.eosinofilos',
                 'hemograma.eosinofilos.percentual',
                 'hemograma.leucocitos',
                 'hemograma.linfocitos',
                 'hemograma.linfocitos.percentual',
                 'hemograma.monocitos',
                 'hemograma.monocitos.percentual',
                 'hemograma.neutrofilos',
                 'hemograma.rdw',
                 'hemograma.volume.plaquetario.medio',
                 'proteina.c.reativa.plasma.proteina.creativa',
                 'sodio.sodio',
                 'ureia.plasma.ureia',
                 'dimero.d.plasma.dimeros.d.quantitativo',
                 'potassio.potassio') -> dados_dir

#aplica a função a cada coluna
dados_dir %>% map(~ re_extremos_dir(.)) %>% as.data.frame -> dados_dir

#renomeia
names(dados_dir) <-c('creatinina.plasma.creatinina',
                 'hemograma.basofilos',
                 'hemograma.basofilos.percentual',
                 'hemograma.eosinofilos',
                 'hemograma.eosinofilos.percentual',
                 'hemograma.leucocitos',
                 'hemograma.linfocitos',
                 'hemograma.linfocitos.percentual',
                 'hemograma.monocitos',
                 'hemograma.monocitos.percentual',
                 'hemograma.neutrofilos',
                 'hemograma.rdw',
                 'hemograma.volume.plaquetario.medio',
                 'proteina.c.reativa.plasma.proteina.creativa',
                 'sodio.sodio',
                 'ureia.plasma.ureia',
                 'dimero.d.plasma.dimeros.d.quantitativo',
                    'potassio.potassio')


In [ ]:
#variáveis com histograma dislocado à direita, remover valores extremos da esquerda
dados %>% select('hemograma.concentracao.de.hemoglobina.corpuscular',
                 'hemograma.eritrocitos',
                 'hemograma.hematocrito',
                 'hemograma.hemoglobina',                 
                 'hemograma.neutrofilos.percentual' ) -> dados_esq

#aplica a função a cada coluna
dados_esq %>% map(~ re_extremos_esq(.)) %>% as.data.frame -> dados_esq

#renomeia
names(dados_esq) <- c('hemograma.concentracao.de.hemoglobina.corpuscular',
                     'hemograma.eritrocitos',
                     'hemograma.hematocrito',
                     'hemograma.hemoglobina',                 
                     'hemograma.neutrofilos.percentual')

In [ ]:
#variáveis com histograma centralizado, remover extremos da direita e da esquerda
dados %>% select('hemograma.hemoglobina.corpuscular.media', 'hemograma.vcm') -> dados_dir_esq

#aplica a função a cada coluna
dados_dir_esq %>% map(~ re_extremos_dir_esq(.)) %>% as.data.frame -> dados_dir_esq

#renomeia
names(dados_dir_esq) <- c('hemograma.hemoglobina.corpuscular.media','hemograma.vcm')

In [ ]:
#variáveis que não tiveram remoção de extremos
dados %>% select("sexo", "gravidade") -> dados_2

#une as variáveis novamente
dados_2 %>% cbind(dados_dir) %>% cbind(dados_esq) %>% cbind(dados_dir_esq) -> dados
dados %>% dim
dados %>% count(gravidade)

In [ ]:
#nomes das variáveis
dados %>% colnames

#visualiza dos dados
dados %>% head

#coloca gravidade como última coluna
dados %>% select(gravidade) -> gravidade
dados %>% select(-gravidade) -> dado
cbind(dado,gravidade) -> dados


In [ ]:
#renomeia as colunas
names(dados) <- (c('Sex', 'Creatinine','Basophils',
                   'Basophils(%)','Eosinophils', 'Eosinophils(%)',
                   'Leukocytes','Lymphocytes', 'Lymphocytes(%)',
                   'Monocytes', 'Monocytes(%)','Neutrophils',
                   'RDW', 'MPV', 'C-reative protein',
                   'Sodium', 'Urea', 'D-dimers',
                   'Potassium','CHC', 'Erythrocytes', 
                   'Hematocrit', 'Hemoglobin', 'Neutrophils(%)', 
                   'MCH', 'MCV', 'Severity'))

In [ ]:
#escreve um arquivo com os dados processados
dados %>% write_csv("bp.csv")